In [25]:
from PIL import Image
import pandas as pd
import numpy as np
import os
import tensorflow as tf

from tensorflow.python.framework import random_seed

import numpy as np
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [26]:
train_labels = pd.read_csv("train_data/train_labels.csv")['invasive'].values



In [27]:
def load_images(indices):
    images= np.zeros(shape=(len(indices), 866, 1154, 3), dtype=np.uint8)
    for p, i in enumerate(indices):
        im = Image.open("train_data/"+str(i+1)+".jpg")
    # open image file and store in variable `im`, then
        images[p] = np.array(im)
    images = images.astype(np.float32)
    images = np.multiply(images, 1.0 / 255.0)

    return images



In [28]:
class DataSet(object):

  def __init__(self,
               image_indices,
               labels,
               seed=None):
    seed1, seed2 = random_seed.get_seed(seed)
    # If op level seed is not set, use whatever graph level seed is returned
    np.random.seed(seed1 if seed is None else seed2)
    self._num_examples = len(image_indices)
    self._image_indices = image_indices
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0

  @property
  def image_indices(self):
    return self._image_indices

  @property
  def labels(self):
    return self._labels

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_completed(self):
    return self._epochs_completed

  def load_images(indices):

        images= np.zeros(shape=(len(indices), 866, 1154, 3), dtype=np.uint8)
        for i in indices:
            im = Image.open("train_data/"+str(i+1)+".jpg")
        # open image file and store in variable `im`, then
            images[i] = np.array(im)
        images = images.astype(np.float32)
        images = np.multiply(images, 1.0 / 255.0)

        return images

  def next_batch(self, batch_size, shuffle=True):
    """Return the next `batch_size` examples from this data set."""
    start = self._index_in_epoch
    # Shuffle for the first epoch
    if self._epochs_completed == 0 and start == 0 and shuffle:
      perm0 = np.arange(self._num_examples)
      np.random.shuffle(perm0)
      self._image_indices = self.image_indices[perm0]
      self._labels = self.labels[perm0]
    # Go to the next epoch
    if start + batch_size > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Get the rest examples in this epoch
      rest_num_examples = self._num_examples - start
      images_rest_part = self._image_indices[start:self._num_examples]
      labels_rest_part = self._image_indices[start:self._num_examples]
      # Shuffle the data
      if shuffle:
        perm = np.arange(self._num_examples)
        np.random.shuffle(perm)
        self._image_indices = self.image_indices[perm]
        self._labels = self.labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size - rest_num_examples
      end = self._index_in_epoch
      images_new_part = self._image_indices[start:end]
      labels_new_part = self._labels[start:end]
      return load_images(np.concatenate((images_rest_part, images_new_part), axis =0)), np.concatenate((labels_rest_part, labels_new_part), axis=0)
    else:
      self._index_in_epoch += batch_size
      end = self._index_in_epoch
      return load_images(self._image_indices[start:end]), self._labels[start:end]

   
        

In [29]:
y_train = train_labels[:1800]
y_dev = train_labels[1800:]
train_data = DataSet(np.arange(0,1800), y_train)
test_data = DataSet(np.arange(1800,len(train_labels)), y_dev)

y_validation = train_labels[1800:]
validation_data = DataSet(np.arange(1800,len(train_labels)), y_validation)

In [30]:
height = 866
width = 1154
channels = 3
n_inputs = height * width

conv1_fmaps = 64
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"
conv2_dropout_rate = 0.25


pool3_fmaps = conv2_fmaps

n_fc1 = 32
fc1_dropout_rate = 0.5

n_outputs = 2

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
    X_resized = tf.image.resize_images(X, [400, 300])
    X_resized_flat = tf.reshape(X_resized,shape=[-1,400* 300*channels])

    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')
    
    X_nn = tf.placeholder(tf.float32, shape=[None, 400* 300* channels], name="X_nn")

'''
conv1 = tf.layers.conv2d(X_resized, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    ##Why are we reshaping the tensor 
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 200 * 150])
    ## The drop outs is not used --> does tensorflow regularise automatically if the below is specified
    pool3_flat_drop = tf.layers.dropout(pool3_flat, conv2_dropout_rate, training=training)
  

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")
    #where do you use drop out layer
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
'''

with tf.name_scope("fc1_nn"):
    fc1_nn = tf.layers.dense(X_resized_flat, 400, activation=tf.nn.relu, name="fc1_nn")
    #where do you use drop out layer
    fc1_drop = tf.layers.dropout(fc1_nn, fc1_dropout_rate, training=training)

with tf.name_scope("output_nn"):
    logits_nn = tf.layers.dense(fc1_nn, n_outputs, name="output_nn")
    Y_proba = tf.nn.softmax(logits_nn, name="Y_proba")
    
'''
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
''' 
with tf.name_scope("train_nn"):
    xentropy_nn = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_nn, labels=y)
    loss_nn = tf.reduce_mean(xentropy_nn)
    optimizer_nn = tf.train.AdamOptimizer()
    training_op_nn = optimizer.minimize(loss_nn)
'''
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
'''

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits_nn, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


In [31]:
X_val, y_val = validation_data.next_batch(80)

In [32]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

In [36]:
n_epochs = 10
batch_size = 30

best_loss_val = np.infty
check_interval = 5
checks_since_last_progress = 0
max_checks_without_progress = 8
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(train_data.num_examples // batch_size):
            X_batch, y_batch = train_data.next_batch(batch_size)
            #X_batch =X_batch.reshape(height*width*channels)
            sess.run(training_op_nn, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss_nn.eval(feed_dict={X: X_val, y: y_val})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val,y: y_val})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    save_path = saver.save(sess, "./my_isd_model")

Epoch 0, train accuracy: 76.6667%, valid. accuracy: 67.5000%, valid. best loss: 1.452171
Epoch 1, train accuracy: 60.0000%, valid. accuracy: 41.2500%, valid. best loss: 1.047881
Epoch 2, train accuracy: 66.6667%, valid. accuracy: 70.0000%, valid. best loss: 1.047881
Early stopping!


In [37]:
#Evaluation
test_batch_size = 80
acc_test =[]
with tf.Session() as sess:
    saver.restore(sess,"./my_isd_model")
    for iteration in range(test_data.num_examples // test_batch_size):
        X_test, y_test = test_data.next_batch(test_batch_size)
        acc_test.append(accuracy.eval(feed_dict={X: X_test, y:y_test}))


print("Test accuracy:", sum(acc_test)/len(acc_test))




Test accuracy: 0.695833345254
